This notebook covers some core operations with Series and DataFrames. Again, this is a quick review to ensure we're all on the same page.

1. Can you use numpy ufuncs (e.g. `np.exp`) on DataFrames?
2. What happens when you add two DataFrames with different indices?

If you know the answers already, you can skip to our first [application](#Application:-Economic-Timeseries). Maybe download some additional datasets from [FRED](https://research.stlouisfed.org) and play with those; try to figure out how to fix the economy. You've got about 15 minutes.

`DataFrame`s and `Series` support all the usual math operations. Additionally,
numpy [`ufuncs`](http://docs.scipy.org/doc/numpy/reference/ufuncs.html) (universal function, e.g. `np.log`) can  be used as expected.

In [ ]:
import numpy as np
import pandas as pd

from dkf2 import side_by_side  # came with the notebooks

# `ufuncs`

A universal function (or `ufunc` for short) is a function that operates on ndarrays in an element-by-element fashion. That is, a `ufunc` is a “vectorized” wrapper for a function that takes a fixed number of scalar inputs and produces a fixed number of scalar outputs.

They can be used just like you'd expect on a `Series` or `DataFrame`.

In [ ]:
np.random.seed(42)
df = pd.DataFrame(np.random.uniform(0, 10, size=(3, 3)))
df

In [ ]:
df + 1

In [ ]:
df ** 2

In [ ]:
np.log(df)

For any decent-sized `DataFrame`, using a `ufunc` will be much faster than applying the function to each row in a for loop or by using `DataFrame.apply`.

# Alignment

When you do binary operations between two `numpy` arrays, like `+`, the arrays are in some sense aligned by position.
It's up to you to ensure that the correct values are in the same position across arrays.

In pandas, we rely on row and column *labels* to do the alignment for us.
Pandas automatically aligns by label when doing operations between `DataFrames` and `Series`.

In [ ]:
df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df2 = pd.DataFrame({'A': [2, 4, 6], 'B': [1, 4, 9]})

side_by_side(df1, df2, "df1", "df2")

In [ ]:
df1 + df2

In [ ]:
# Note the index order
df2 = pd.DataFrame({'A': [6, 2, 4], 'B': [9, 1, 4]}, index=[2, 0, 1])
side_by_side(df1, df2, 'df1', 'df2')

In [ ]:
df1 + df2

In [ ]:
# Different index entirely
df3 = pd.DataFrame(np.random.randn(5, 3), columns=['A', 'B', 'C'])
side_by_side(df1, df3, 'df1', 'df3')

In [ ]:
df1 + df3

# Recap

- `ufuncs` work just fine
- Operations **align by label** and then are performed

# Application: Economic Timeseries

In [ ]:
from itertools import chain

import pandas as pd

import seaborn as sns
import statsmodels.api as sm
from pandas_datareader import data

import matplotlib.pyplot as plt
from toolz import partitionby

%matplotlib inline
pd.options.display.max_rows = 10
sns.set_style('ticks')

Pandas recently split off some web-based data reading functionality into its own package `pandas-datareader`.<br> Earlier, I used it to grab data from [FRED](http://research.stlouisfed.org) like so:

```python
gdp = data.DataReader("GDP", data_source='fred',
                      start='1929', end='2014').squeeze()
cpi = data.DataReader("CPIAUCSL", data_source='fred',
                      start='1947-01', end='2015-05').squeeze()
rec = data.DataReader('USREC', data_source='fred',
                      start='1854-12-01',
                      end='2014-08-01').squeeze()
gdp.to_csv('data/gdp.csv', header=True)
cpi.to_csv('data/cpi.csv', header=True)
rec.to_csv('data/rec.csv', header=True)
```

In [ ]:
with open('data/gdp.csv') as f:
    print(''.join(f.readlines(100)))

In [ ]:
pd.read_csv?

In [ ]:
gdp = pd.read_csv('data/gdp.csv', index_col='DATE', parse_dates=['DATE']).squeeze()
cpi = pd.read_csv('data/cpi.csv', index_col='DATE', parse_dates=['DATE']).squeeze()
rec = pd.read_csv('data/rec.csv', index_col='DATE', parse_dates=['DATE']).squeeze()

In [ ]:
gdp.head(n=5)

In [ ]:
gdp.tail()

In [ ]:
cpi.head()

In [ ]:
rec.head()

## Methods

DataFrames and Series have a bunch of useful aggregation methods, `.mean`, `.max`, `.std`, etc.

In [ ]:
gdp.mean()

## Plotting

DataFrames and Series can make decent looking plots with a few lines of code.

I wanted to incude bars for recession indicators.
The only pandas-relavent bit is `Series.iteritems`, which you *almost* never want to use since it's relatively slow.

In [ ]:
# Retrieves the next item from this iterator
next(rec.iteritems())  

In [ ]:
groups = partitionby(lambda x: x[1] == 1, rec.iteritems())
recessions = filter(lambda x: x[0][1] == 1, groups)
spans = [(months[0][0], months[-1][0]) for months in recessions]

In [ ]:
def add_rec_bars(ax=None):
    ax = ax or plt.gca()
    ylim = ax.get_ylim()
    for span in spans:
        ax.fill_between(span, *ax.get_ylim(), color='k', alpha=.25)
    ax.set_ylim(*ylim)
    return ax

sns.set(style="white", context="talk")
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
ax = gdp.plot(title='GDP', linewidth=3)
add_rec_bars(ax)
sns.despine()

Let's put some of those operations to use.

<div class="alert alert-success">
    <b>EXERCISE 1</b>: Convert CPI to base 2009
</div>

CPI is the Consumer Price Index. The Index part just means that it doesn't really have any units,
just some (arbitrary) time span that is set to 100, and every other observation is relative to that time.
The CPI we have is indexed to the average over 1982-1984.

**Understanding the CPI**<br />
A Consumer Price Index of 158 indicates 58% inflation since 1982, while a CPI index of 239 would indicate 139% inflation since 1982. The commonly quoted inflation rate of say 3% is actually the change in the Consumer Price Index from a year earlier. By looking at the change in the Consumer Price Index we can see that an item that cost an average of 9.9 cents in 1913 would cost us<br />
+ 1.82 in 2003
+ 2.02 in 2007
+ 2.33 in 2013
+ 2.39 in 2016 

**Convert the CPI base-2009 by dividing the entire Series `cpi` by the average CPI in 2009.**

Try breaking the problem into pieces:
1. Select just the rows from 2009 (`.loc`)
  + Timeseries (dates in the index) have [special rules](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#datetimeindex-partial-string-indexing) for slicing. Pass in a string with the subset you want
    + `.loc['2010-01-01']` select all observations from that day
    + `.loc['2010-01']` selects all observations from that month
2. calculate the average on those rows
3. divide `cpi` by 100 * that number (assign that to `cpi09`)

In [ ]:
# Your code goes here
# Assign the result to cpi09

In [ ]:
# %load solutions/solutions_cpi.py
# CPI (base 2009)


In [ ]:
# Let's plot
ax = cpi09.plot(title='CPI-2009')
add_rec_bars(ax=ax)
sns.despine();

<div class="alert alert-success">
    <b>EXERCISE 2</b>: Calculate Real GDP with base 2009
</div>


Next, try calculating real GDP, with 2009 as the base year.
This is defined to be `gdp` divided by the base-2009 CPI for that time period.

Call the result `rgdp` and try plotting it.

In [ ]:
# Your code goes here
rgdp = ...

In [ ]:
# Your plot here
ax = ...

If you have trouble with the plotting, look into the `.fillna` and `.dropna` methods.
The next section explains what's going on.

If you finish this exercise early, check out the "Extra Exercises" section at the bottom of the notebook.

# Real GDP: Alignment

Let's say we want real GDP (adjusted for inflation).

\begin{equation}
    rGDP_t = \frac{GDP_t}{CPI_t}
\end{equation}

Problem: our CPI is monthly but GDP is quarterly. Also the two Series have different start and end points. If you didn't have automatic label alignment, you'd have to jump through hoops to select the correct subset of each series.


```python
# this is boolean indexing, we'll see more later
gdp / cpi09[(cpi.index.month % 3 == 1) & (cpi09.index.year <= 2014)]
```

... Hint: The process above was unneccesary. Pandas built-in operations will automatically align for you.

In [ ]:
rgdp = (gdp / cpi09)
rgdp

The `NaN`s are missing value indicators. `NaN`s can crop up for many reasons, but in this case it's because the labels didn't overlap perfectly.

Binary operations (addition, multiplication, ...) consists of two steps.

1. align by label
2. operation

In [ ]:
l, r = gdp.align(cpi09)
side_by_side(l.head(), r.head())


While binary operations *propogate* missing values, aggregations like `sum` or `mean`, will ignore them.
Pandas provides methods for detecting missing data `.isnull`, filling missing data `.fillna`,
or you can dropping it, which is what I'll do here.

In [ ]:
rgdp.dropna()

Pandas methods are non-mutating by default. This means that even though I called `.dropna()` above, `rgdp` still has missing values.

In [ ]:
rgdp

To capture the change, assign a name to the result. In this case, I just the same name `rgdp`.

In [ ]:
real_gdp = rgdp.dropna()
real_gdp.head()

In [ ]:
ax = gdp.pct_change().plot(
    figsize=(15, 5), label='GDP', legend=True, linewidth=2
)
real_gdp.pct_change().plot(
    ax=ax, label='real GDP', legend=True, linewidth=2
)
add_rec_bars(ax)
sns.despine()

# Recap

DataFrames allow you to rapidly explore data, iterating on ideas.

- `ufunc`s work as expected
- operations between pandas objects automatically align on labels

<div class="alert alert-info">
    <b>Extra Exercises</b>
</div>

Explore [FRED](https://fred.stlouisfed.org) for additional datasets.

See if you can produce an updated version of this graph from [*Calculated Risk*](Calculated Risk: BLS: Jobs Openings at 5.0 million in December, Up 28% Year-over-year).

In [ ]:
from IPython.display import Image
Image(url='figures/JOLTSDec2014.PNG')

This shows the components of turnover in the labor market. People leave jobs for two reasons: 1. Quits (light blue) or 2. Layoffs, discharges, or other (red).

Companies post job openings (yellow) and fill some number of those (Blue).

The difference between the openings and hires represents the change in the stock of open positions, and measures the slackness of the labor market.

The difference between the Hires and the sum of Quites and Layoffs / Discharges represents the change in the employed labor force (though the unemployment rate is calculated differently).

To get you started, the FRED series codes are

- [JTSJOL](https://fred.stlouisfed.org/series/JTSJOL): Openings
- [JTSQUL](https://fred.stlouisfed.org/series/JTSQUL): Quits
- [JTSHIL](https://fred.stlouisfed.org/series/JTSHIL): Hires
- [JTSLDL](https://fred.stlouisfed.org/series/JTSLDL): Layoffs

Use the `pandas_datareader.data.DataReader` class, which accepts a list of symbols, and a `data_source` ('`fred'`).

As a hint, you can use pandas' `.plot.area()` method for the Quits and Layoffs series. That handles all the stacking for you.

In [ ]:
# Your code here


In [ ]:
# Plotting Hint
ax = res[['quits', 'layoffs']].plot.area()
res[['hires', 'openings']].plot(ax=ax)